## 3. Bitcoin y su relación con algunas commodities

Acontinuación se plantea un modelo multilineal para ver la relación entre el precio del bitcoin y el oro,la plata y el petroleo.

---

#### Creación del data set
Acontinuación se crea el data set, tomando los precios del bitcoin, el oro, la plata y el petróleo durante el último año

In [1]:
using CSV
using DataFrames

#Bitcoin Data
dataBitcoin = CSV.read("actualBitcoin.csv",DataFrame);
dataBitcoin.Date .= dataBitcoin.Date;
dataBitcoin.PriceFloat = dataBitcoin.Price;

#Oil Data
dataOil = CSV.read("actualOil.csv",DataFrame);
dataOil.Date .= dataOil.Date;
dataOil.PriceFloat = dataOil.Price;

#Gold Data
dataGold = CSV.read("actualGold.csv",DataFrame);
dataGold.Date .= dataGold.Date;
dataGold.PriceFloat = dataGold.Price;

#Silver Data
dataSilver = CSV.read("actualSilver.csv",DataFrame);
dataSilver.Date .= dataSilver.Date;
dataSilver.PriceFloat = dataSilver.Price;

#Date in common
common=intersect(dataSilver.Date,dataOil.Date,dataGold.Date,dataBitcoin.Date)

#restrict the data
dataOil=dataOil[in(common).(dataOil.Date), :];
dataBitcoin=dataBitcoin[in(common).(dataBitcoin.Date), :];
dataGold=dataGold[in(common).(dataGold.Date), :];
dataSilver=dataSilver[in(common).(dataSilver.Date), :];


In [2]:
#Parse the Data
dataSilver.PriceFloat = dataSilver.Price;

dataOil.PriceFloat = dataOil.Price;

dataGold.Price .= replace.(dataGold.Price, "," => "");
dataGold.PriceFloat = parse.(Float64, dataGold.Price);

dataBitcoin.Price .= replace.(dataBitcoin.Price, "," => "");
dataBitcoin.PriceFloat = parse.(Float64, dataBitcoin.Price);


Antes de plantear el modelo veamos el comportamiento del precio de las commodities y del bitcoin mediante el siguiente gráfico

In [4]:
using Dates,DataFrames
#convert string databitcoin.Date to type Date
dataBitcoin[!, :Date] = Date.(dataBitcoin.Date, dateformat"uuu d, yyyy");

In [42]:
using PlotlyJS
#plots de cada uno de los precios
plot1=plot(dataBitcoin.Date,dataBitcoin.PriceFloat,Layout(title="Precio del bitcoin"))
plot2=plot(dataBitcoin.Date,dataGold.PriceFloat,Layout(title="Precio del oro"))
plot3=plot(dataBitcoin.Date,dataSilver.PriceFloat,Layout(title="Precio de la plata"))
plot4=plot(dataBitcoin.Date,dataOil.PriceFloat,Layout(title="Precio del petróleo"))
plott=[plot1;plot2;plot3;plot4]
relayout!(plott, height=1000, width=800, title_text="Precios")
restyle!(plott,name=["btc","gold","silver","oil"])
plott

data: [
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis",
  "scatter with fields name, type, x, xaxis, y, and yaxis"
]

layout: "layout with fields annotations, height, margin, template, title, width, xaxis1, xaxis2, xaxis3, xaxis4, yaxis1, yaxis2, yaxis3, and yaxis4"

de antemano podemos observar que no se ve una relación lineal entre el gráfico de las commodities y el del bitcoin

#### Modelo
Utilizaremos el siguiente modelo multilineal que será ajustado mediante mínimos cuadrados
$$\beta_0+\beta_1X_1+\beta_2X_2+\beta_3X_3=b$$
donde $X_1,X_2,X_3$ serán los precios del oro, la plata y el petróleo respectivamente, $\beta_0$ será nuestro intersecto con el eje y, y los $\beta_i$ nos indicarán la relación del respectivo comoditie con el precio del bitcoin, y $b$ es el precio del bitcoin.

In [12]:
#Creación del modelo
N = size(dataBitcoin.PriceFloat,1);
A = [ones(263,1)   dataGold.PriceFloat[10:272] dataSilver.PriceFloat[10:272] dataOil.PriceFloat[10:272] ];
b = dataBitcoin.PriceFloat[10:272];

# los beta_i ajustados del modelo
x = A\b

4-element Vector{Float64}:
 171495.51308583308
    -67.77194467031661
   -106.08574266788405
     15.571072587267484

Realizando la predicción para una semana obtenemos la siguiente gráfica:

In [28]:

#predicción y plot de la predicción y el precio real
pred=ones(9,1).*x[1]+x[3].*dataSilver.PriceFloat[1:9]+x[4].*dataOil.PriceFloat[1:9]+x[2].*dataGold.PriceFloat[1:9]
result=plot(dataBitcoin.Date[1:9],[pred dataBitcoin.PriceFloat[1:9]])
restyle!(result, name=["prediccion","precio real"])
result

[38670.125037196005; 40369.98513877175; 39790.02609878944; 39712.01850696895; 38687.08687333015; 40963.8123586351; 39548.76058180058; 37528.503045478545; 35895.38525302871;;]

data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with fields margin and template"

Comparando la predicción con el precio real durante el última año obtendríamos la siguiente gráfica

In [36]:
pred=ones(272,1).*x[1]+x[3].*dataSilver.PriceFloat+x[4].*dataOil.PriceFloat+x[2].*dataGold.PriceFloat
result=plot(dataBitcoin.Date,[pred dataBitcoin.PriceFloat])
restyle!(result, name=["prediccion","precio real"])
result

data: [
  "scatter with fields name, type, x, and y",
  "scatter with fields name, type, x, and y"
]

layout: "layout with fields margin and template"

de esta manera el modelo quedaría ajustado a:
$$171495.513-67.771PriceGold-106.085PriceSilver+15.571PriceOil=PriceBitcoin$$
y según el modelo se tendría entonces que por cada dolar de incremento del precio del Oro, disminuye en promedio 67.7 dolares el precio del bitcoin; por cada dolar de incremento del precio de la plata, disminuye 106 dolares en promedio el precio del bitcoin; y por cada dolar de incremento del precio del Petróleo, aumenta 15.57 dolares el precio del bitcoin; lo anterior teniendo en cuenta que el intersecto en $y$ está en 171495.513, es decir que si el precio de las comodities estuviera en cero, entonces el precio del bitcoin sería 171495.513 dolares. Sin embargo de acuerdo a las gráficas no habría una relación lineal del precio de las 3 commodities con el precio del bitcoin.